In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pickle
# import optuna
import time
import sys
import torch
import os
import argparse
import random
from neural_nets import SimpleLSTM, SimpleMLP, SimpleGRU, SimpleTransformer


In [2]:
on_cluster = True

from load_data_funs import load_data, gen_batch_data_fixations_choice, gen_batch_data_fixations_only, gen_batch_data_choice_only

# set path to the human and simulation data
if on_cluster:
    sim_data_path = '/scratch/gpfs/erussek/RNN_project/optimal_fixation_sims'
    human_data_path = '/scratch/gpfs/erussek/RNN_project/human_trials.json'
else:
    sim_data_path = '/Users/erussek/Dropbox/Griffiths_Lab_Stuff/Data/RNNs/optimal_fixation_sims'
    human_data_path = '/Users/erussek/Dropbox/Griffiths_Lab_Stuff/Data/RNNs/human_trials.json'



In [3]:
# compute a model which is proportion of fixation time thus far into the trial...
%run load_data_funs
# so this will be from fixation data only...
run_idx = 0
train_data_sim, val_data_sim, test_data_sim, train_data_human, val_data_human, test_data_human = load_data(sim_data_path, human_data_path,this_seed=run_idx,split_human_data=True)
use_human_data=True


In [4]:
%run load_data_funs

train_seq_part = 'fix_and_choice'
fix_unit = 'all'
# Functions to generate data
gen_data_func_by_train_seq_part = {"fix_and_choice": gen_batch_data_fixations_choice, "fix_only": gen_batch_data_fixations_only, "choice_only": gen_batch_data_choice_only}
gen_data_func_pre = gen_data_func_by_train_seq_part[train_seq_part]
# set the fix unit
gen_data_func = lambda a, b, c, use_human_data=False : gen_data_func_pre(a, b, c, fix_unit = fix_unit, use_human_data = use_human_data)

In [5]:
fix_unit = 'all'
use_human_data = False
batch_size = 32
batch_idx = 0
data = train_data_sim

# filter list of trials that are in this batch
batch_sim_data = data[batch_idx*batch_size:((batch_idx+1)*(batch_size))]

# all sequences in the batch, attended item is coded as idx (as 0, 1, 2)
if use_human_data:
    batch_fixation_sequences_idx = [(np.array(trial_data['fixations'])-1).tolist() for trial_data in batch_sim_data]
else:
    batch_fixation_sequences_idx = [trial_data['fixations'] for trial_data in batch_sim_data]

batch_choices_idx = [trial_data['choice'] - 1 for trial_data in batch_sim_data]

# first 3 are fixation, last is choice...
batch_sequences_cat = [np.array([to_categorical(x, num_classes = 3) for x in this_sequence]) for this_sequence in batch_fixation_sequences_idx]

# pick a better name for this
if fix_unit != 'ID':
    batch_sequences_cat = [compute_sum_fixations(this_seq, fix_unit = fix_unit) for this_seq in batch_sequences_cat]

# append zeros horizontally
batch_sequences_cat = [np.hstack((batch_sequences_cat[i], np.zeros((batch_sequences_cat[i].shape[0],3)))) if len(batch_sequences_cat[i]) > 0 else np.array([]) for i in range(len(batch_sequences_cat))]

if fix_unit == 'all':
    num_tokens = 12
else:
    num_tokens = 6

# now append to each of these the choice info - the choice gets it's own channel (of 3)
batch_sequences_cat_w_choices = [  np.vstack((batch_sequences_cat[i], [to_categorical(num_tokens - 3 + batch_choices_idx[i], num_classes = num_tokens)])) if len(batch_sequences_cat[i]) > 0 else np.array([]) for i in range(len(batch_sequences_cat))]

batch_data = pad_sequences(batch_sequences_cat_w_choices)
batch_data = batch_data.astype('float32')

batch_targets_init = np.array([trial_data['value'] for trial_data in batch_sim_data], dtype = 'float32')
batch_targets_cont = [np.repeat([batch_targets_init[i]],len(batch_sequences_cat_w_choices[i]),axis=0) for i in range(len(batch_sequences_cat_w_choices))]

batch_targets = pad_sequences(batch_targets_cont)
batch_targets = batch_targets.astype('float32')

In [32]:
batch_data[0]

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 1., 0., ..., 0., 0., 0.],
       [0., 1., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 1., 0.]], dtype=float32)

In [23]:
batch_sequences_cat = [np.hstack((batch_sequences_cat[i], np.zeros((batch_sequences_cat[i].shape[0],3)))) if len(batch_sequences_cat[i]) > 0 else np.array([]) for i in range(len(batch_sequences_cat))]


In [17]:
print(len(batch_sequences_cat[10]))

0


In [20]:
i = 9
np.hstack((batch_sequences_cat[i], np.zeros((batch_sequences_cat[i].shape[0],3))))

array([[1., 0., 0., 0., 0., 0.],
       [0., 0., 1., 0., 0., 0.],
       [0., 0., 1., 0., 0., 0.],
       [0., 0., 1., 0., 0., 0.],
       [0., 0., 1., 0., 0., 0.],
       [0., 0., 1., 0., 0., 0.],
       [0., 0., 1., 0., 0., 0.],
       [0., 1., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0., 0.],
       [0., 0., 1., 0., 0., 0.],
       [0., 0., 1., 0., 0., 0.],
       [0., 0., 1., 0., 0., 0.],
       [0., 0., 1., 0., 0., 0.]])

In [44]:
fix_seqs = gen_data_func(32,0,train_data_sim, use_human_data = False)

ValueError: all the input array dimensions for the concatenation axis must match exactly, but along dimension 1, the array at index 0 has size 3 and the array at index 1 has size 12

In [45]:
batch_sequences_cat[0][0]

array([0., 1., 0., 0., 1., 0., 0., 1., 0., 0., 0., 0.])

In [46]:
batch_sequences_cat_w_choices[0][0]

array([0., 1., 0., 0., 1., 0., 0., 1., 0., 0., 1., 0.])

In [64]:
s.shape

(1, 16, 3)

In [6]:
def compute_sum_fixations(this_seq, return_type = 'sum'):
    this_seq = np.array(batch_sequences_cat[0])
    sum_fix = np.cumsum(this_seq,0)
    total_fix = np.sum(sum_fix,1)
    prop_fix = np.array([sum_fix[i,:] / total_fix[i] for i in range(len(this_seq))])
    
    if return_type == 'sum':
        return sum_fix
    
    elif return_type == 'prop':
        return prop_fix
    else:
        display('surprise')

return_type = 'sum'
    
from sequential_tasks import pad_sequences, to_categorical

batch_size = 32
batch_idx = 5
data = train_data_sim
use_human_data=True
# filter list of trials that are in this batch
batch_sim_data = data[batch_idx*batch_size:((batch_idx+1)*(batch_size))]

# all sequences in the batch, attended item is coded as idx (as 0, 1, 2)
if use_human_data:
    batch_fixation_sequences_idx = [(np.array(trial_data['fixations'])-1).tolist() for trial_data in batch_sim_data]
else:
    batch_fixation_sequences_idx = [trial_data['fixations'] for trial_data in batch_sim_data]

# first 3 are fixation, last is choice...
batch_sequences_cat = [[to_categorical(x, num_classes = 3) for x in this_sequence] for this_sequence in batch_fixation_sequences_idx]

# pick a better name for this
if return_type != 'flat':
    batch_sequences_cat = [compute_sum_fixations(this_seq, return_type = return_type) for this_seq in batch_sequences_cat]
    
# now append to each of these the choice info - the choice gets it's own channel (of 3)
batch_data = pad_sequences(batch_sequences_cat)
batch_data = batch_data.astype('float32')

batch_targets_init = np.array([trial_data['value'] for trial_data in batch_sim_data], dtype = 'float32')
batch_targets_cont = [np.repeat([batch_targets_init[si]],len(batch_sequences_cat[i]),axis=0) for i in range(len(batch_sequences_cat))]

batch_targets = pad_sequences(batch_targets_cont)
batch_targets = batch_targets.astype('float32')

NameError: name 'si' is not defined

In [ ]:
batch_data